In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.grid_search import GridSearchCV,RandomizedSearchCV
import matplotlib.pyplot as plt

C:\Users\user\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [9]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv',header =None)


In [10]:
#for training set
buying_onehot = pd.get_dummies(train.buying_price)
buying_onehot.columns = ['f01','f02','f03','f04']
train = pd.concat([train,buying_onehot],axis  = 1)
maintainence_onehot = pd.get_dummies(train.maintainence_cost)
maintainence_onehot.columns =['f05','f06','f07','f08']
train = pd.concat([train,maintainence_onehot],axis = 1)
luggage_onehot = pd.get_dummies(train.luggage_boot_size)
luggage_onehot.columns =['f09','f10','f11']
train = pd.concat([train,luggage_onehot],axis = 1)
safety_onehot = pd.get_dummies(train.safety_rating)
safety_onehot.columns =['f12','f13','f14']
train = pd.concat([train,safety_onehot],axis = 1)

#for test set
buying_onehot = pd.get_dummies(test.buying_price)
buying_onehot.columns = ['f01','f02','f03','f04']
test = pd.concat([test,buying_onehot],axis  = 1)
maintainence_onehot = pd.get_dummies(test.maintainence_cost)
maintainence_onehot.columns =['f05','f06','f07','f08']
test = pd.concat([test,maintainence_onehot],axis = 1)
luggage_onehot = pd.get_dummies(test.luggage_boot_size)
luggage_onehot.columns =['f09','f10','f11']
test = pd.concat([test,luggage_onehot],axis = 1)
safety_onehot = pd.get_dummies(test.safety_rating)
safety_onehot.columns =['f12','f13','f14']
test = pd.concat([test,safety_onehot],axis = 1)



AttributeError: 'DataFrame' object has no attribute 'buying_price'

In [13]:
train.buying_price

0       3
1       3
2       1
3       4
4       3
5       2
6       1
7       3
8       1
9       2
10      3
11      3
12      4
13      1
14      2
15      4
16      4
17      3
18      3
19      1
20      4
21      4
22      3
23      2
24      4
25      1
26      4
27      3
28      1
29      4
       ..
1598    2
1599    1
1600    2
1601    2
1602    1
1603    4
1604    3
1605    3
1606    2
1607    2
1608    1
1609    4
1610    2
1611    1
1612    2
1613    2
1614    3
1615    2
1616    4
1617    1
1618    2
1619    4
1620    1
1621    4
1622    3
1623    2
1624    4
1625    4
1626    3
1627    3
Name: buying_price, Length: 1628, dtype: int64

In [ ]:
train =train.drop(['buying_price','maintainence_cost','luggage_boot_size','safety_rating'],axis=1)
test =test.drop(['buying_price','maintainence_cost','luggage_boot_size','safety_rating'],axis=1)
del buying_onehot , maintainence_onehot,luggage_onehot,safety_onehot

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(train.drop(['popularity'],axis=1),train.popularity,
                                                 test_size=0.33, random_state=42)

In [ ]:
#xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
#       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
#       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
#       objective='multi:softmax', reg_alpha=0, reg_lambda=1,
#       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [ ]:
xg_train = xgb.DMatrix(X_train,label =y_train)
xg_test = xgb.DMatrix(X_test,label=y_test)
param ={}

param['objective'] = 'multi:softmax'
param['eta'] = 0.1
param['max_depth'] = 3
param['silent'] = 1
param['nthread'] = 4


param['num_class'] = 5
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 1720
bst = xgb.train(param, xg_train, num_round, watchlist)
#bst = xgb.train(param, xgb.DMatrix(train.drop(['popularity'],axis=1),label = train.popularity), num_round, watchlist)
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != y_test) / y_test.shape[0]
print('Test error using softmax = {}'.format(error_rate))

In [ ]:
train.columns

In [161]:
param['objective'] = 'multi:softprob'
num_round = 1720
param['silent'] = 0
bst2 = xgb.train(param, xg_train, num_round, watchlist)
predproba = bst2.predict(xgb.DMatrix(test))


[0]	train-merror:0.166055	test-merror:0.178439
[1]	train-merror:0.166055	test-merror:0.178439
[2]	train-merror:0.166055	test-merror:0.178439
[3]	train-merror:0.149541	test-merror:0.167286
[4]	train-merror:0.149541	test-merror:0.167286
[5]	train-merror:0.149541	test-merror:0.167286
[6]	train-merror:0.143119	test-merror:0.152416
[7]	train-merror:0.143119	test-merror:0.152416
[8]	train-merror:0.144037	test-merror:0.156134
[9]	train-merror:0.144037	test-merror:0.156134
[10]	train-merror:0.143119	test-merror:0.152416
[11]	train-merror:0.143119	test-merror:0.152416
[12]	train-merror:0.144037	test-merror:0.156134
[13]	train-merror:0.144037	test-merror:0.156134
[14]	train-merror:0.143119	test-merror:0.152416
[15]	train-merror:0.144037	test-merror:0.156134
[16]	train-merror:0.137615	test-merror:0.152416
[17]	train-merror:0.133945	test-merror:0.141264
[18]	train-merror:0.126606	test-merror:0.141264
[19]	train-merror:0.116514	test-merror:0.130112
[20]	train-merror:0.114679	test-merror:0.130112
[2

[170]	train-merror:0.005505	test-merror:0.035316
[171]	train-merror:0.005505	test-merror:0.035316
[172]	train-merror:0.005505	test-merror:0.037175
[173]	train-merror:0.005505	test-merror:0.037175
[174]	train-merror:0.005505	test-merror:0.037175
[175]	train-merror:0.00367	test-merror:0.035316
[176]	train-merror:0.00367	test-merror:0.033457
[177]	train-merror:0.00367	test-merror:0.033457
[178]	train-merror:0.00367	test-merror:0.033457
[179]	train-merror:0.00367	test-merror:0.033457
[180]	train-merror:0.00367	test-merror:0.033457
[181]	train-merror:0.00367	test-merror:0.035316
[182]	train-merror:0.00367	test-merror:0.035316
[183]	train-merror:0.002752	test-merror:0.035316
[184]	train-merror:0.002752	test-merror:0.033457
[185]	train-merror:0.002752	test-merror:0.033457
[186]	train-merror:0.002752	test-merror:0.033457
[187]	train-merror:0.002752	test-merror:0.033457
[188]	train-merror:0.002752	test-merror:0.033457
[189]	train-merror:0.002752	test-merror:0.033457
[190]	train-merror:0.001835	

[362]	train-merror:0	test-merror:0.024164
[363]	train-merror:0	test-merror:0.024164
[364]	train-merror:0	test-merror:0.024164
[365]	train-merror:0	test-merror:0.024164
[366]	train-merror:0	test-merror:0.024164
[367]	train-merror:0	test-merror:0.024164
[368]	train-merror:0	test-merror:0.024164
[369]	train-merror:0	test-merror:0.024164
[370]	train-merror:0	test-merror:0.024164
[371]	train-merror:0	test-merror:0.024164
[372]	train-merror:0	test-merror:0.024164
[373]	train-merror:0	test-merror:0.024164
[374]	train-merror:0	test-merror:0.024164
[375]	train-merror:0	test-merror:0.024164
[376]	train-merror:0	test-merror:0.024164
[377]	train-merror:0	test-merror:0.024164
[378]	train-merror:0	test-merror:0.024164
[379]	train-merror:0	test-merror:0.024164
[380]	train-merror:0	test-merror:0.024164
[381]	train-merror:0	test-merror:0.024164
[382]	train-merror:0	test-merror:0.024164
[383]	train-merror:0	test-merror:0.024164
[384]	train-merror:0	test-merror:0.024164
[385]	train-merror:0	test-merror:0

[558]	train-merror:0	test-merror:0.020446
[559]	train-merror:0	test-merror:0.020446
[560]	train-merror:0	test-merror:0.020446
[561]	train-merror:0	test-merror:0.020446
[562]	train-merror:0	test-merror:0.020446
[563]	train-merror:0	test-merror:0.020446
[564]	train-merror:0	test-merror:0.020446
[565]	train-merror:0	test-merror:0.020446
[566]	train-merror:0	test-merror:0.020446
[567]	train-merror:0	test-merror:0.020446
[568]	train-merror:0	test-merror:0.020446
[569]	train-merror:0	test-merror:0.020446
[570]	train-merror:0	test-merror:0.020446
[571]	train-merror:0	test-merror:0.020446
[572]	train-merror:0	test-merror:0.020446
[573]	train-merror:0	test-merror:0.020446
[574]	train-merror:0	test-merror:0.020446
[575]	train-merror:0	test-merror:0.020446
[576]	train-merror:0	test-merror:0.020446
[577]	train-merror:0	test-merror:0.020446
[578]	train-merror:0	test-merror:0.020446
[579]	train-merror:0	test-merror:0.020446
[580]	train-merror:0	test-merror:0.018587
[581]	train-merror:0	test-merror:0

[754]	train-merror:0	test-merror:0.020446
[755]	train-merror:0	test-merror:0.020446
[756]	train-merror:0	test-merror:0.020446
[757]	train-merror:0	test-merror:0.020446
[758]	train-merror:0	test-merror:0.020446
[759]	train-merror:0	test-merror:0.020446
[760]	train-merror:0	test-merror:0.020446
[761]	train-merror:0	test-merror:0.020446
[762]	train-merror:0	test-merror:0.020446
[763]	train-merror:0	test-merror:0.020446
[764]	train-merror:0	test-merror:0.020446
[765]	train-merror:0	test-merror:0.020446
[766]	train-merror:0	test-merror:0.020446
[767]	train-merror:0	test-merror:0.020446
[768]	train-merror:0	test-merror:0.020446
[769]	train-merror:0	test-merror:0.020446
[770]	train-merror:0	test-merror:0.020446
[771]	train-merror:0	test-merror:0.020446
[772]	train-merror:0	test-merror:0.020446
[773]	train-merror:0	test-merror:0.020446
[774]	train-merror:0	test-merror:0.020446
[775]	train-merror:0	test-merror:0.020446
[776]	train-merror:0	test-merror:0.020446
[777]	train-merror:0	test-merror:0

[950]	train-merror:0	test-merror:0.020446
[951]	train-merror:0	test-merror:0.020446
[952]	train-merror:0	test-merror:0.020446
[953]	train-merror:0	test-merror:0.020446
[954]	train-merror:0	test-merror:0.020446
[955]	train-merror:0	test-merror:0.020446
[956]	train-merror:0	test-merror:0.020446
[957]	train-merror:0	test-merror:0.020446
[958]	train-merror:0	test-merror:0.020446
[959]	train-merror:0	test-merror:0.020446
[960]	train-merror:0	test-merror:0.020446
[961]	train-merror:0	test-merror:0.020446
[962]	train-merror:0	test-merror:0.020446
[963]	train-merror:0	test-merror:0.020446
[964]	train-merror:0	test-merror:0.020446
[965]	train-merror:0	test-merror:0.020446
[966]	train-merror:0	test-merror:0.020446
[967]	train-merror:0	test-merror:0.020446
[968]	train-merror:0	test-merror:0.020446
[969]	train-merror:0	test-merror:0.020446
[970]	train-merror:0	test-merror:0.020446
[971]	train-merror:0	test-merror:0.020446
[972]	train-merror:0	test-merror:0.020446
[973]	train-merror:0	test-merror:0

[1142]	train-merror:0	test-merror:0.018587
[1143]	train-merror:0	test-merror:0.018587
[1144]	train-merror:0	test-merror:0.018587
[1145]	train-merror:0	test-merror:0.018587
[1146]	train-merror:0	test-merror:0.018587
[1147]	train-merror:0	test-merror:0.018587
[1148]	train-merror:0	test-merror:0.018587
[1149]	train-merror:0	test-merror:0.018587
[1150]	train-merror:0	test-merror:0.018587
[1151]	train-merror:0	test-merror:0.018587
[1152]	train-merror:0	test-merror:0.018587
[1153]	train-merror:0	test-merror:0.018587
[1154]	train-merror:0	test-merror:0.018587
[1155]	train-merror:0	test-merror:0.018587
[1156]	train-merror:0	test-merror:0.018587
[1157]	train-merror:0	test-merror:0.018587
[1158]	train-merror:0	test-merror:0.018587
[1159]	train-merror:0	test-merror:0.018587
[1160]	train-merror:0	test-merror:0.018587
[1161]	train-merror:0	test-merror:0.018587
[1162]	train-merror:0	test-merror:0.018587
[1163]	train-merror:0	test-merror:0.018587
[1164]	train-merror:0	test-merror:0.018587
[1165]	trai

[1333]	train-merror:0	test-merror:0.018587
[1334]	train-merror:0	test-merror:0.018587
[1335]	train-merror:0	test-merror:0.018587
[1336]	train-merror:0	test-merror:0.018587
[1337]	train-merror:0	test-merror:0.018587
[1338]	train-merror:0	test-merror:0.018587
[1339]	train-merror:0	test-merror:0.018587
[1340]	train-merror:0	test-merror:0.018587
[1341]	train-merror:0	test-merror:0.018587
[1342]	train-merror:0	test-merror:0.018587
[1343]	train-merror:0	test-merror:0.018587
[1344]	train-merror:0	test-merror:0.018587
[1345]	train-merror:0	test-merror:0.018587
[1346]	train-merror:0	test-merror:0.018587
[1347]	train-merror:0	test-merror:0.018587
[1348]	train-merror:0	test-merror:0.018587
[1349]	train-merror:0	test-merror:0.018587
[1350]	train-merror:0	test-merror:0.018587
[1351]	train-merror:0	test-merror:0.018587
[1352]	train-merror:0	test-merror:0.018587
[1353]	train-merror:0	test-merror:0.018587
[1354]	train-merror:0	test-merror:0.018587
[1355]	train-merror:0	test-merror:0.018587
[1356]	trai

[1524]	train-merror:0	test-merror:0.018587
[1525]	train-merror:0	test-merror:0.018587
[1526]	train-merror:0	test-merror:0.018587
[1527]	train-merror:0	test-merror:0.018587
[1528]	train-merror:0	test-merror:0.018587
[1529]	train-merror:0	test-merror:0.018587
[1530]	train-merror:0	test-merror:0.018587
[1531]	train-merror:0	test-merror:0.018587
[1532]	train-merror:0	test-merror:0.018587
[1533]	train-merror:0	test-merror:0.018587
[1534]	train-merror:0	test-merror:0.018587
[1535]	train-merror:0	test-merror:0.018587
[1536]	train-merror:0	test-merror:0.018587
[1537]	train-merror:0	test-merror:0.018587
[1538]	train-merror:0	test-merror:0.018587
[1539]	train-merror:0	test-merror:0.018587
[1540]	train-merror:0	test-merror:0.018587
[1541]	train-merror:0	test-merror:0.018587
[1542]	train-merror:0	test-merror:0.018587
[1543]	train-merror:0	test-merror:0.018587
[1544]	train-merror:0	test-merror:0.018587
[1545]	train-merror:0	test-merror:0.018587
[1546]	train-merror:0	test-merror:0.018587
[1547]	trai

[1715]	train-merror:0	test-merror:0.018587
[1716]	train-merror:0	test-merror:0.018587
[1717]	train-merror:0	test-merror:0.018587
[1718]	train-merror:0	test-merror:0.018587
[1719]	train-merror:0	test-merror:0.018587


In [81]:
temp=pd.DataFrame(bst.predict(xgb.DMatrix(test)))
#temp.to_csv('prediction4.csv',index = False)

In [150]:

model = MLPClassifier(activation='relu', alpha=0.0001, batch_size=32, beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(20000,), learning_rate='invscaling',
       learning_rate_init=0.001, max_iter=20000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [151]:
model.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size=32, beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(20000,), learning_rate='invscaling',
       learning_rate_init=0.001, max_iter=20000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [190]:
predprobaNN = model.predict_proba(test)

In [153]:
model.classes_

array([1, 2, 3, 4], dtype=int64)

In [154]:
error_rate = np.sum(y != y_test) / y_test.shape[0]

In [155]:
error_rate

0.033457249070631967

In [179]:
predprobaNN + predproba

ValueError: Unable to coerce to DataFrame, shape must be (100, 4): given (100, 5)

In [174]:
xgb=pd.DataFrame(predproba).drop([0],axis=1)
predprobaNN=pd.DataFrame(predprobaNN)